In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# %cd /content/drive/MyDrive/AI/task3/train_recognition/
!ls

Mounted at /content/drive
drive  sample_data


In [ ]:
!unzip -q data.zip -d datasets/nto

In [ ]:
!pip install gdown -q
!curl 'https://storage.yandexcloud.net/datasouls-competitions/ai-nto-final-2022/data.zip' -H 'authority: storage.yandexcloud.net' -H 'sec-ch-ua: " Not A;Brand";v="99", "Chromium";v="98", "Google Chrome";v="98"' -H 'sec-ch-ua-mobile: ?0' -H 'sec-ch-ua-platform: "Windows"' -H 'upgrade-insecure-requests: 1' -H 'user-agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36' -H 'accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9' -H 'sec-fetch-site: none' -H 'sec-fetch-mode: navigate' -H 'sec-fetch-user: ?1' -H 'sec-fetch-dest: document' -H 'accept-language: en-US,en;q=0.9' --output 'data.zip'
!unzip -q data.zip -d datasets/nto
!gdown 'https://drive.google.com/u/0/uc?id=1uMXAAf1iWy7zDqUlWVUS2CqFp8FrH526&export=download&confirm=t' -q
!unzip ocr.zip
!gdown 'https://drive.google.com/u/0/uc?id=1XdDlbuOGaxodhFU_Bo9-zoZBguJE8oZD&export=download&confirm=t'
!unzip forms.zip -d datasets/
!gdown 'https://drive.google.com/uc?id=19zRCcRvTT3Oe72E4Kq4SL0SquGTpMDpO&export=download&confirm=t'
!unrar x HKR_Dataset_Words_Public.rar datasets/ -pP@ssw0rd-HKR2020 
!mv datasets/20200923_Dataset_Words_Public datasets/hkr
!gdown 'https://drive.google.com/uc?id=1s4793xuwJNp1W6d9ifqQLaGB3GpHuuE3&confirm=t'
!unzip -q archive.zip -d datasets/kaggle

Streaming output truncated to the last 5000 lines.
Extracting  datasets/20200923_Dataset_Words_Public/img/8_36_839.jpg       91%  OK 
Extracting  datasets/20200923_Dataset_Words_Public/img/8_36_841.jpg       91%  OK 
Extracting  datasets/20200923_Dataset_Words_Public/img/8_36_842.jpg       91%  OK 
Extracting  datasets/20200923_Dataset_Words_Public/img/8_36_843.jpg       91%  OK 
Extracting  datasets/20200923_Dataset_Words_Public/img/8_36_844.jpg       91%  OK 
Extracting  datasets/20200923_Dataset_Words_Public/img/8_36_845.jpg       91%  OK 
Extracting  datasets/20200923_Dataset_Words_Public/img/8_36_846.jpg       91%  OK 
Extracting  datasets/20200923_Dataset_Words_Public/img/8_36_847.jpg       91%  OK 
Extracting  datasets/20200923_Dataset_Words_Public/img/8_36_848.jpg       91%  OK 
Extracting  datasets/20200923_Dataset_Words_Public/img/8_36_849.jpg       91%  OK 
Extracting  datasets/20200923

In [ ]:
from IPython.display import clear_output
from tqdm import tqdm
import os
import time
import torch
import numpy as np
import json
import random
!pip install wandb -q
import wandb

from ocr.utils import (
    val_loop, load_pretrain_model, FilesLimitControl, AverageMeter, sec2min
)

from ocr.dataset import get_data_loader
from ocr.transforms import get_train_transforms, get_val_transforms
from ocr.tokenizer import Tokenizer
from ocr.config import Config
from ocr.models import CRNN

seed = 0xC0FFEE
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

     |████████████████████████████████| 1.7 MB 5.3 MB/s 
     |████████████████████████████████| 181 kB 73.2 MB/s 
     |████████████████████████████████| 144 kB 77.7 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 


In [ ]:
def train_loop(data_loader, model, criterion, optimizer, epoch, scheduler):
    loss_avg = AverageMeter()
    strat_time = time.time()
    model.train()
    tqdm_data_loader = tqdm(data_loader, total=len(data_loader), leave=False)
    for images, texts, enc_pad_texts, text_lens in tqdm_data_loader:
        model.zero_grad()
        images = images.to(DEVICE)
        batch_size = len(texts)
        output = model(images)
        output_lenghts = torch.full(
            size=(output.size(1),),
            fill_value=output.size(0),
            dtype=torch.long
        )
        loss = criterion(output, enc_pad_texts, output_lenghts, text_lens)
        # print(output.shape)
        loss_avg.update(loss.item(), batch_size)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 2)
        optimizer.step()
        scheduler.step()
    loop_time = sec2min(time.time() - strat_time)
    for param_group in optimizer.param_groups:
        lr = param_group['lr']
    train_stats = {
        'loss': loss_avg.avg,
        'lr': lr,
    }
    print(f'\nEpoch {epoch}, Loss: {loss_avg.avg:.5f}, '
          f'LR: {lr:.7f}, loop_time: {loop_time}')
    return train_stats


def get_loaders(tokenizer, config):
    train_transforms = get_train_transforms(
        height=config.get_image('height'),
        width=config.get_image('width'),
        prob=0.2
    )
    train_loader = get_data_loader(
        transforms=train_transforms,
        csv_paths=config.get_train_datasets('csv_path'),
        tokenizer=tokenizer,
        dataset_probs=config.get_train_datasets('prob'),
        epoch_size=config.get_train('epoch_size'),
        batch_size=config.get_train('batch_size'),
        drop_last=True
    )
    val_transforms = get_val_transforms(
        height=config.get_image('height'),
        width=config.get_image('width')
    )
    val_loader = get_data_loader(
        transforms=val_transforms,
        csv_paths=config.get_val_datasets('csv_path'),
        tokenizer=tokenizer,
        dataset_probs=config.get_val_datasets('prob'),
        epoch_size=config.get_val('epoch_size'),
        batch_size=config.get_val('batch_size'),
        drop_last=False
    )
    return train_loader, val_loader

In [ ]:
import pandas as pd
cyr_train_raw = pd.read_csv('/content/datasets/kaggle/train.tsv',sep='\t',names=['filename','text'])
cyr_test_raw = pd.read_csv('/content/datasets/kaggle/test.tsv',sep='\t',names=['filename','text'])
cyr_raw = pd.concat([cyr_train_raw, cyr_test_raw], ignore_index=True)
cyr_data = {}
for a,b in cyr_raw.iterrows():
    if '\n' in b.values[1]:
        continue
    if 'test' in b.values[0]:
        cyr_data[os.path.join('/content/datasets/kaggle/test', b.values[0])] = b.values[1]
    else:
        cyr_data[os.path.join('/content/datasets/kaggle/train', b.values[0])] = b.values[1]
    # print(b.values)
train_cyr = [(k, v) for k, v in cyr_data.items()]
print('train len', len(train_cyr))

split_coef = 0.75
train_len = int(len(train_cyr)*split_coef)

train_cyr_splitted = train_cyr[:train_len]
val_cyr_splitted = train_cyr[train_len:]

print('train len after split', len(train_cyr_splitted))
print('val len after split', len(val_cyr_splitted))

train len 31739
train len after split 23804
val len after split 7935


In [ ]:
import pandas as pd
forms_raw = pd.read_csv('/content/datasets/forms/data.csv')
forms_data = {}
for a,b in forms_raw.iterrows():
    forms_data[os.path.join('/content/datasets/forms/images', b.values[0])] = b.values[1]
    # print(b.values)
train_forms = [(k, v) for k, v in forms_data.items()]
print('train len', len(train_forms))

split_coef = 0.75
train_len = int(len(train_forms)*split_coef)

train_forms_splitted = train_forms[:train_len]
val_forms_splitted = train_forms[train_len:]

print('train len after split', len(train_forms_splitted))
print('val len after split', len(val_forms_splitted))

train len 5202
train len after split 3901
val len after split 1301


In [ ]:
import os
from tqdm import tqdm

hkr_ann = os.listdir('/content/datasets/hkr/ann')

hkr_annos = {}
for anno in hkr_ann:
    with open('/content/datasets/hkr/ann/' + anno, 'r') as f:
        hkr_annos['/content/datasets/hkr/img/' + anno.split('.')[0] + '.jpg'] = json.load(f)['description']
# with open('/content/datasets/hkr/hkr.json', 'r') as f:
#     hkr_annos = json.load(f)      
train_hkr = [(k, v) for k, v in hkr_annos.items()]
random.Random(seed).shuffle(train_hkr)
print('train len', len(train_hkr))

split_coef = 0.75
train_len = int(len(train_hkr)*split_coef)

train_hkr_splitted = train_hkr[:train_len]
val_hkr_splitted = train_hkr[train_len:]

print('train len after split', len(train_hkr_splitted))
print('val len after split', len(val_hkr_splitted))

train len 64943
train len after split 48707
val len after split 16236


In [ ]:
import json
with open('/content/drive/MyDrive/ИИ2021/FINAL/data/annotations_val.json') as f:
    annots_val = json.load(f)
with open('/content/drive/MyDrive/ИИ2021/FINAL/data/annotations_train.json') as f:
    annots_train = json.load(f)

In [ ]:
import pandas as pd
df = pd.read_csv('datasets/nto/data/train_recognition/labels.csv')

val_data = {}
train_data = {}
for a, b in df.iterrows():
    parent_name = b.values[2]
    if 'eng' in parent_name:
        continue
    id = b.values[0].split('.')[0].split('_')[0]
    for i in range(len(annots_val['images'])):
        _parent_name = annots_val['images'][i]['file_name']
        if _parent_name == parent_name:
            val_data[os.path.join('/content/datasets/nto/data/train_recognition/images', b.values[0])] = b.values[1]
            break

for a, b in df.iterrows():
    parent_name = b.values[2]
    if 'eng' in parent_name:
        continue
    id = b.values[0].split('.')[0].split('_')[0]
    for i in range(len(annots_train['images'])):
        _parent_name = annots_train['images'][i]['file_name']
        if _parent_name == parent_name:
            train_data[os.path.join('/content/datasets/nto/data/train_recognition/images', b.values[0])] = b.values[1]
            break

train_data = [(k, v) for k, v in train_data.items()]
val_data = [(k, v) for k, v in val_data.items()]

In [ ]:
# import cv2
# for file_name, _ in tqdm(train_data):
#     # print(file_name)
#     img = cv2.imread(file_name)
#     if not len(img.shape) == 3:
#         # os.remove(file_name)
#         print(file_name)

In [ ]:
train_data_splitted = train_data
train_data_splitted += train_forms_splitted
train_data_splitted += train_cyr_splitted
train_data_splitted += train_hkr_splitted
random.Random(seed).shuffle(train_data_splitted)
val_data_splitted = val_data
val_data_splitted += val_forms_splitted
val_data_splitted += val_cyr_splitted
val_data_splitted += val_hkr_splitted
random.Random(seed).shuffle(val_data_splitted)

print('train len after split', len(train_data_splitted))
print('val len after split', len(val_data_splitted))

import csv
with open('/content/train.csv', 'w') as csvfile:
    w = csv.writer(csvfile)
    w.writerow(['filename', 'text'])
    for name, text in train_data_splitted:
        w.writerow([name, text])
        # if 'forms' in name or 'cyr' in name or 'hkr' in name:
        #     w.writerow([name, text])
        # else:
        #     w.writerow([os.path.join('images', name), text])
        
with open('/content/val.csv', 'w') as csvfile:
    w = csv.writer(csvfile)
    w.writerow(['filename', 'text'])
    for name, text in val_data_splitted:
        w.writerow([name, text])
        # if 'forms' in name or 'cyr' in name or 'hkr' in name:
        #     w.writerow([name, text])
        # else:
        #     w.writerow([os.path.join('images', name), text])
            
print('data len ', len(train_data_splitted + val_data_splitted))

train len after split 223778
val len after split 37609
data len  261387


In [ ]:
with open('/content/drive/MyDrive/ИИ2021/FINAL/weights&configs/ocr_config_infin.json') as f:
    cfg = json.load(f) 
# run = wandb.init(project="Task", entity="nto", name='crnn_hkr40', config=cfg, reinit=True)

In [ ]:
config = Config('/content/drive/MyDrive/ИИ2021/FINAL/weights&configs/ocr_config_infin.json')
tokenizer = Tokenizer(config.get('alphabet'))
os.makedirs(config.get('save_dir'), exist_ok=True)
train_loader, val_loader = get_loaders(tokenizer, config)

model = CRNN(number_class_symbols=tokenizer.get_num_chars())
if config.get('pretrain_path'):
    print(1)
    states = load_pretrain_model(config.get('pretrain_path'), model)
    model.load_state_dict(states)
    print('Load pretrained model')
model.to(DEVICE)

criterion = torch.nn.CTCLoss(blank=0, reduction='mean', zero_infinity=True)
optimizer = torch.optim.AdamW(model.parameters(), lr=config.get('lr'),
                              weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer=optimizer,
    epochs=config.get('num_epochs'),
    steps_per_epoch=len(train_loader),
    max_lr=config.get('max_lr'),
    pct_start=0.1,
    anneal_strategy='cos',
    final_div_factor=10 ** 5
)
# if config.get('pretrain_path'):
#   for i in range(20):
#     for i in tqdm(train_loader):
#       scheduler.step()
# weight_limit_control = FilesLimitControl()
best_cer = np.inf

# val_stats = val_loop(val_loader, model, criterion, tokenizer, DEVICE)
for epoch in range(0, config.get('num_epochs')):
    train_stats = train_loop(train_loader, model, criterion, optimizer,
                          epoch, scheduler)
    
    val_stats = val_loop(val_loader, model, criterion, tokenizer, DEVICE)
    
    val_cer = val_stats['cer']
    val_loss = val_stats['loss']
    if val_cer < best_cer:
        best_cer = val_cer
        model_save_path = os.path.join(
            config.get('save_dir'), f'{val_cer:.4f}-{val_loss}-Epoch{epoch}.ckpt')
        torch.save(model.state_dict(), model_save_path)
        print('Model weights saved')
#         weight_limit_control(model_save_path)
#     run.log({
#         'val_cer': val_cer,
#         'val_loss': val_loss,
#         'train_loss': train_stats['loss'],
#         'lr': train_stats['lr']
#     })
# run.finish()

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

1
['.config', 'drive', 'data.zip', 'ocr', 'archive.zip', 'val.csv', 'HKR_Dataset_Words_Public.rar', 'datasets', 'ocr.zip', 'train.csv', 'forms.zip', 'sample_data']
Load pretrained model


  0%|          | 0/3496 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
100%|██████████| 3496/3496 [02:09<00:00, 27.02it/s]



Epoch 20, Loss: 0.13477, LR: 0.0000750, loop_time: 85m 19s


...

Validation, cer: 0.0470, acc: 0.7748, wer: 0.1897, loss: 0.1601, loop_time: 6m 11s



Epoch 28, Loss: 0.08836, LR: 0.0000010, loop_time: 85m 17s


Validation, cer: 0.0466, acc: 0.7761, wer: 0.1885, loss: 0.1587, loop_time: 6m 9s



Epoch 29, Loss: 0.08808, LR: 0.0000000, loop_time: 85m 15s


Validation, cer: 0.0465, acc: 0.7764, wer: 0.1880, loss: 0.1585, loop_time: 6m 11s
Model weights saved


In [ ]:
Validation, cer: 0.0877, acc: 0.6197, wer: 0.3299, loss: 0.2919, loop_time: 6m 13s
Validation, cer: 0.0862, acc: 0.6216, wer: 0.3274, loss: 0.2875, loop_time: 6m 18s